In [ ]:
import PIL.Image as Image
import cv2
import numpy as np
import os

import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

import pathlib                               


In [ ]:
train_dir = pathlib.Path("./dog_cat_dataset/training_set")
test_dir=pathlib.Path("./dog_cat_dataset/testing_set")
image_size = 200  # We'll resize input images to this size
num_epochs = 15  # For real training, use num_epochs=100. 10 is a test value

print(train_dir)
print(test_dir)


In [ ]:
cat_train= list(train_dir.glob('cats/*'))
dog_train= list(train_dir.glob('dogs/*'))

print("Cat length: ", len(cat_train))
print("Dog length: ", len(dog_train))

In [ ]:
Image.open(str(cat_train[1245]))

In [ ]:
Image.open(str(dog_train[1569]))


In [ ]:
images_dict = {
    'cats': cat_train,    
    'dogs': dog_train,
}
labels_dict = {
    'cats': 0,
    'dogs': 1,
    
}

In [ ]:
images_dict["dogs"][:5]

In [ ]:
img= cv2.imread(str(images_dict['cats'][0]))

In [ ]:
img.shape

In [ ]:
X, y = [], []
for name,images in images_dict.items():#returns key , path for each record
    for image in images:
        if '_DS_Store' in str(image):
            print(f"Skipping _DS_Store file: {image}")
            continue
        if '.DS_Store' in str(image):
            print(f"Skipping .DS_Store file: {image}")
            continue
        img=cv2.imread(str(image))
        if img is not None:
            resize_img = cv2.resize(img, (image_size, image_size))
            X.append(resize_img)
            y.append(labels_dict[name])
        else:
            print(f"Unable to load image: {image}")


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
print(X.shape)
print(y)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, Y_train, x_label, y_label = train_test_split(X, y, test_size=0.20 ,random_state=4)

In [ ]:
print(X_train.shape)
print(x_label.shape)
print(Y_train.shape)
print(y_label.shape)

In [ ]:
train_images = X_train / 255.0

test_images = Y_train / 255.0


In [ ]:
print(X_train[0])


In [ ]:
print(train_images[0])


In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
class_names = ['cats', 'dogs']

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):    
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[x_label[i]])
plt.show()

In [ ]:
model = Sequential()

# Input Layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape = (image_size,image_size,3))) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Bloack 1 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# Block 2
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# Block 3
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fully Connected layers 
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2,activation='softmax'))
model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
print(train_images.shape)
print(x_label.shape)

In [ ]:
model.fit(train_images, x_label , epochs=num_epochs)


In [ ]:
test_loss, test_acc = model.evaluate(test_images, y_label, verbose=2)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)


In [ ]:
predictions[0]


In [ ]:
print(predictions[0])
print("model: ", np.argmax(predictions[12]))
print("actual: ", y_label[12])


In [ ]:
print(predictions.shape)
print(y_label.shape)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(2))
  plt.yticks([])
  thisplot = plt.bar(range(2), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_label, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  y_label)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_label, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  y_label)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_label, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_label)
plt.tight_layout()
plt.show()

In [ ]:
cat_test= list(test_dir.glob('cats/*'))
dog_test= list(test_dir.glob('dogs/*'))

print("Cat length: ", len(cat_test))
print("Dog length: ", len(dog_test))

In [ ]:
predict_index = 45
# Grab an image from the test dataset.
image = cat_test[predict_index]
img = cv2.imread(str(image))

resize_img = cv2.resize(img, (image_size, image_size))
resize_img = resize_img / 255.0

plt.figure()
plt.imshow(resize_img)
plt.colorbar()
plt.grid(True)
plt.xlabel(class_names[0])
plt.show()


In [ ]:
# Add the image to a batch where it's the only member.
resize_img = (np.expand_dims(resize_img,0))

print(resize_img.shape)

In [ ]:
predictions_single = probability_model.predict(resize_img)
predictions_label = class_names[np.argmax(predictions_single[0])]

print(predictions_single)
print(predictions_label)


In [ ]:
probability_model.save("./models/cnn.keras")
